<table>
<tr>
<td>V1 </td>
<td>Simple feature engineering</td>
<td>second</td>
</tr>
<tr>
<td>V2 </td>
<td>Add pressure and inefficiency</td>
<td>second</td>
</tr>    
</table>



| Tables   |      Are      |  Cool |
|----------|:-------------:|------:|
| col 1 is |  left-aligned | $1600 |
| col 2 is |    centered   |   $12 |
| col 3 is | right-aligned |    $1 |

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import joblib 
import os
import sklearn 

from load_data import load_data_from_csv

In [2]:
from data_preprocessor import CompositeDataPreprocessor
from feature_engineering import EnrichDFDataPreprocessor, RemoveIrrelevantFeaturesDataPreprocessor, DropTargetNADataPreprocessor

In [3]:
from model_pipeline.k_fold_pipeline import ManualKFoldModelPipeline
from model_pipeline.lgb_pipeline import LGBModelPipelineFactory
from model_pipeline.xgb_pipeline import XGBModelPipelineFactory
from model_pipeline.cbt_pipeline import CatBoostModelPipelineFactory

## Data preprocessing pipeline

In [4]:
processors = [
    EnrichDFDataPreprocessor(),
    DropTargetNADataPreprocessor(),
    RemoveIrrelevantFeaturesDataPreprocessor(['stock_id', 'date_id','time_id', 'row_id'])    
]
processor = CompositeDataPreprocessor(processors)

### Load data

In [5]:
# DATA_PATH = '/kaggle/input'
DATA_PATH = '..'
df_train, df_test, revealed_targets, sample_submission = load_data_from_csv(DATA_PATH)
print(df_train.columns)

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id'],
      dtype='object')


### Pre-process data

In [6]:
df_train = processor.apply(df_train)
print(df_train.shape[0])
print(df_train.columns)
display(df_train.tail())

5237892
Index(['seconds_in_bucket', 'imbalance_size', 'imbalance_buy_sell_flag',
       'reference_price', 'matched_size', 'far_price', 'near_price',
       'bid_price', 'bid_size', 'ask_price', 'ask_size', 'wap', 'target',
       'imb_s1', 'imb_s2', 'reference_price_ask_price_bid_price_imb2',
       'reference_price_ask_price_wap_imb2',
       'reference_price_bid_price_wap_imb2', 'ask_price_bid_price_wap_imb2',
       'pressure', 'inefficiency'],
      dtype='object')


,seconds_in_bucket,imbalance_size,imbalance_buy_sell_flag,reference_price,matched_size,far_price,near_price,bid_price,bid_size,ask_price,...,wap,target,imb_s1,imb_s2,reference_price_ask_price_bid_price_imb2,reference_price_ask_price_wap_imb2,reference_price_bid_price_wap_imb2,ask_price_bid_price_wap_imb2,pressure,inefficiency
5237975,540,2440722.89,-1,1.000317,28280361.74,0.999734,0.999734,1.000317,32257.04,1.000434,...,1.000328,2.310276,-0.816784,-0.841104,5.269212e+11,9.636364e+00,NaN,9.636364,75.664813,0.086305
5237976,540,349510.47,-1,1.000643,9187699.11,1.000129,1.000386,1.000643,205108.40,1.000900,...,1.000819,-8.220077,0.374254,-0.926706,NaN,4.602273e-01,7.926335e+11,0.460227,1.704028,0.038041
5237977,540,0.00,0,0.995789,12725436.10,0.995789,0.995789,0.995789,16790.66,0.995883,...,0.995797,1.169443,-0.829388,-1.000000,-2.822256e+11,1.075000e+01,NaN,10.750000,0.000000,0.000000
5237978,540,1000898.84,1,0.999210,94773271.05,0.999210,0.999210,0.998970,125631.72,0.999210,...,0.999008,-1.540184,-0.684154,-0.979099,-9.251859e-13,-1.099231e-12,5.315789e+00,5.315789,1.494117,0.010561
5237979,540,1884285.71,-1,1.002129,24073677.32,1.000859,1.001494,1.002129,250081.44,1.002447,...,1.002274,-6.530285,-0.091024,-0.854820,NaN,1.193103e+00,-6.530219e+11,1.193103,7.534688,0.078272


## Train

In [7]:
os.system('mkdir models')

1

In [8]:
N_fold = 5

In [12]:
lgb_model_pipeline = ManualKFoldModelPipeline(LGBModelPipelineFactory(), n_fold=N_fold, dump_model=True)

In [13]:
lgb_model_pipeline.init_model()

In [14]:
# specific set up for ManualKFoldModelPipeline to pass None
lgb_model_pipeline.train(df_train, None)

Training fold 0 - start
Training fold 0 - initialized
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.207050 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4648
[LightGBM] [Info] Number of data points in the train set: 4190313, number of used features: 20
[LightGBM] [Info] Start training from score -0.060201
Training until validation scores don't improve for 100 rounds
Did not meet early stopping. Best iteration is:
[50]	valid_0's l1: 6.40048
Training fold 0 - finished training
Training fold 0 - end
Training fold 1 - start
Training fold 1 - initialized
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.537650 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 4648
[LightGBM] [Info] Number of data points in the train set: 4190313, number of used features: 20
[LightGBM] [Info] Start training from score -0.069737
Trainin

In [15]:
lgb_models = lgb_model_pipeline.get_model()
print(len(lgb_models))

5


# Load

In [ ]:
models = []

In [25]:
def load(modelname, fold):
    models.append(joblib.load(f'models/{modelname}_{fold}.model'))

In [26]:
for i in range (0, 5):
    load('lgb', i)
    # load('xgb', i)
    # load('cbt', i)

In [27]:
models

[LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1')]

# Evaluate

In [28]:
models

[LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1'),
 LGBMRegressor(n_estimators=50, objective='regression_l1')]

In [29]:
df_train_ = enrich_df_with_features(df_train)
df_train_.columns

Index(['stock_id', 'date_id', 'seconds_in_bucket', 'imbalance_size',
       'imbalance_buy_sell_flag', 'reference_price', 'matched_size',
       'far_price', 'near_price', 'bid_price', 'bid_size', 'ask_price',
       'ask_size', 'wap', 'target', 'time_id', 'row_id',
       'reference_price_far_price_imb', 'reference_price_near_price_imb',
       'reference_price_ask_price_imb', 'reference_price_bid_price_imb',
       'reference_price_wap_imb', 'far_price_near_price_imb',
       'far_price_ask_price_imb', 'far_price_bid_price_imb',
       'far_price_wap_imb', 'near_price_ask_price_imb',
       'near_price_bid_price_imb', 'near_price_wap_imb',
       'ask_price_bid_price_imb', 'ask_price_wap_imb', 'bid_price_wap_imb',
       'imb_s1', 'imb_s2', 'reference_price_ask_price_bid_price_imb2',
       'reference_price_ask_price_wap_imb2',
       'reference_price_bid_price_wap_imb2', 'ask_price_bid_price_wap_imb2',
       'pressure', 'inefficiency'],
      dtype='object')

In [30]:
df_train['baseline_prediction'] = 0
baseline_mae = (df_train['baseline_prediction'] - df_train['target']).abs().mean()
print(baseline_mae)

6.40777074811524


In [31]:
def print_prediction(column):    
    mae = (df_train[column] - df_train['target']).abs().mean()
    print(mae, '. MAE improvement in basis points: ', (baseline_mae - mae))

In [32]:
simple_mapping = {
    1: 0.1,
    0: 0,
    -1: -0.1
}
df_train['simple_prediction'] = df_train['imbalance_buy_sell_flag'].map(simple_mapping)
print_prediction('simple_prediction')

6.407056596608261 . MAE improvement in basis points:  0.000714151506978844


In [33]:
df_train['model_prediction'] = np.mean([model.predict(df_train_[features]) for model in models], 0)
print_prediction('model_prediction')

6.299599382342986 . MAE improvement in basis points:  0.10817136577225384


In [34]:
df_train['lgb_prediction'] = np.mean([model.predict(df_train_[features]) for model in models if type(model) == lgb.LGBMRegressor], 0)
print_prediction('lgb_prediction')

6.299599382342986 . MAE improvement in basis points:  0.10817136577225384


In [35]:
df_train['cbt_prediction'] = np.mean([model.predict(df_train_[features]) for model in models if type(model) == cbt.core.CatBoostRegressor], 0)
print_prediction('cbt_prediction')

c:\Users\Florence\anaconda3\envs\optiver2023\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Florence\anaconda3\envs\optiver2023\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan . MAE improvement in basis points:  nan


In [36]:
df_train['xgb_prediction'] = np.mean([model.predict(df_train_[features]) for model in models if type(model) == xgb.sklearn.XGBRegressor], 0)
print_prediction('xgb_prediction')

c:\Users\Florence\anaconda3\envs\optiver2023\lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Florence\anaconda3\envs\optiver2023\lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


nan . MAE improvement in basis points:  nan


In [37]:
type(models[0]) == lgb.LGBMRegressor
type(models[5]) == cbt.core.CatBoostRegressor
type(models[10]) == xgb.sklearn.XGBRegressor

IndexError: list index out of range

In [38]:
type(models[10])

IndexError: list index out of range

In [39]:
df_train[['target', 'baseline_prediction', 'simple_prediction', 'model_prediction']]

,target,baseline_prediction,simple_prediction,model_prediction
0,-3.029704,0,0.1,-0.939437
1,-5.519986,0,-0.1,0.503778
2,-8.389950,0,-0.1,-1.628825
3,-4.010200,0,-0.1,1.896495
4,-7.349849,0,-0.1,-3.192552
...,...,...,...,...
5237975,2.310276,0,-0.1,-1.566881
5237976,-8.220077,0,-0.1,-1.311699
5237977,1.169443,0,0.0,-0.278890
5237978,-1.540184,0,0.1,0.798639


In [40]:
df_train['model_prediction'] = np.mean([model.predict(df_train_[features]) for model in models], 0)

# Submit

In [ ]:
import optiver2023
env = optiver2023.make_env()
iter_test = env.iter_test()

In [ ]:
len(features)

In [ ]:
counter = 0
# sample_prediction['target'] = 0
# env.predict(sample_prediction)
for (test, revealed_targets, sample_prediction) in iter_test:
#     print(test.shape)
    test_ = enrich_df_with_features(test)[features]
#     print(test_.shape)
#     print(len(features))
#     print(len(test_))
    sample_prediction['target'] = np.mean([model.predict(test_) for model in models], 0)
    env.predict(sample_prediction)
    counter += 1